#  Task 1: Uber Data Analysis & ETL Pipeline Development

## 🔹 Task Overview
This task involves **extracting, transforming, and loading (ETL)** Uber trip data into a **structured format** for further analysis. The goal is to clean the raw dataset, extract meaningful features, and organize the data into a **star schema** with multiple dimension tables and a fact table.

## 🔹 Key Objectives
- **Data Cleaning:** Handle missing values and remove duplicates.
- **Feature Extraction:** Convert timestamps and extract useful time-based features.
- **Data Transformation:** Encode categorical variables and normalize numerical data.
- **Schema Design:** Create dimension tables:
  - **Datetime Dimension**
  - **Passenger Count Dimension**
  - **Trip Distance Dimension**
  - **Rate Code Dimension**
  - **Pickup & Dropoff Location Dimensions**
  - **Payment Type Dimension**
- **Fact Table Creation:** Combine all dimensions into a structured table for efficient querying.
- **ETL Implementation:** Process and structure the data using Python and Pandas.
- **Data Storage:** Save the cleaned and transformed dataset for further analysis.

## 🔹 Expected Outcome
- A **cleaned and well-structured dataset** in **star schema format**, making it easier for data analysis and visualization.
- A saved **processed dataset (`processed_uber_data.csv`)** for further use.

---


### Load and Explore the Dataset

In [2]:
import io
import pandas as pd
import requests

In [3]:
# Load the dataset
df = pd.read_csv("uber_data.csv")

# Display first few rows
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


### Convert Date-Time Columns

In [4]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

### Remove Duplicates

In [5]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

### Extract Date-Time Features (Datetime Dimension Table)

In [6]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].copy()

datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday

datetime_dim['datetime_id'] = datetime_dim.index

# Rearranging columns
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]


### Create Passenger Count Dimension Table

In [8]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[['passenger_count_id','passenger_count']]

trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[['trip_distance_id','trip_distance']]

### Create Rate Code Dimension Table

In [9]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_type)
rate_code_dim = rate_code_dim[['rate_code_id','RatecodeID','rate_code_name']]

# rate_code_dim.head()

In [10]:
rate_code_dim.head()


,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,3,Newark


###  Create Location Dimension Tables

In [11]:
pickup_location_dim = df[['pickup_longitude', 'pickup_latitude']].reset_index(drop=True)
pickup_location_dim['pickup_location_id'] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[['pickup_location_id','pickup_latitude','pickup_longitude']] 


dropoff_location_dim = df[['dropoff_longitude', 'dropoff_latitude']].reset_index(drop=True)
dropoff_location_dim['dropoff_location_id'] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[['dropoff_location_id','dropoff_latitude','dropoff_longitude']]

###  Create Payment Type Dimension Table

In [12]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

### Create Fact Table

In [13]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id') \
             .merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id') \
             .merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id') \
             .merge(pickup_location_dim, left_on='trip_id', right_on='pickup_location_id') \
             .merge(dropoff_location_dim, left_on='trip_id', right_on='dropoff_location_id')\
             .merge(datetime_dim, left_on='trip_id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id') \
             [['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'pickup_location_id', 'dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [14]:
payment_type_dim.columns


Index(['payment_type_id', 'payment_type', 'payment_type_name'], dtype='object')

In [15]:
fact_table.columns


Index(['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
       'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
       'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [16]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,1,N,1,1,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,2,2,2,2,N,2,2,2,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,3,2,3,3,3,3,N,3,3,3,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,4,2,4,4,4,4,N,4,4,4,98.0,0.0,0.0,0.00,15.50,0.3,113.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,1,99995,99995,99995,99995,N,99995,99995,99995,5.0,0.0,0.5,0.00,0.00,0.3,5.80
99996,99996,1,99996,99996,99996,99996,N,99996,99996,99996,14.0,0.0,0.5,2.00,0.00,0.3,16.80
99997,99997,1,99997,99997,99997,99997,N,99997,99997,99997,29.0,0.0,0.5,8.80,5.54,0.3,44.14
99998,99998,2,99998,99998,99998,99998,N,99998,99998,99998,5.5,0.5,0.5,1.36,0.00,0.3,8.16


In [17]:
# CREATE OR REPLACE TABLE `data-with-darshil.uber_dataset.tbl_analysis_report` AS (
# SELECT
#   f.VendorID,
#   f.tpep_pickup_datetime,
#   f.tpep_dropoff_datetime,
#   p.passenger_count,
#   td.trip_distance,
#   rc.RatecodeID,
#   f.store_and_fwd_flag,
#   pl.pickup_latitude,
#   pl.pickup_longitude,
#   dl.dropoff_latitude,
#   dl.dropoff_longitude,
#   pt.payment_type,
#   f.fare_amount,
#   f.extra,
#   f.mta_tax,
#   f.tip_amount,
#   f.tolls_amount,
#   f.improvement_surcharge,
#   f.total_amount
# FROM
#   `data-with-darshil.uber_dataset.fact_table` f
#   JOIN `data-with-darshil.uber_dataset.passenger_count_dim` p ON f.passenger_count_id = p.passenger_count_id
#   JOIN `data-with-darshil.uber_dataset.trip_distance_dim` td ON f.trip_distance_id = td.trip_distance_id
#   JOIN `data-with-darshil.uber_dataset.rate_code_dim` rc ON f.rate_code_id = rc.rate_code_id
#   JOIN `data-with-darshil.uber_dataset.pickup_location_dim` pl ON f.pickup_location_id = pl.pickup_location_id
#   JOIN `data-with-darshil.uber_dataset.dropoff_location_dim` dl ON f.dropoff_location_id = dl.dropoff_location_id
#   JOIN `data-with-darshil.uber_dataset.payment_type_dim` pt ON f.payment_type_id = pt.payment_type_id);

In [18]:
print(df.isnull().sum())


VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
pickup_longitude         0
pickup_latitude          0
RatecodeID               0
store_and_fwd_flag       0
dropoff_longitude        0
dropoff_latitude         0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
trip_id                  0
dtype: int64


In [19]:
numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())


In [20]:
categorical_cols = ["store_and_fwd_flag", "payment_type", "VendorID", "RatecodeID"]
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])


In [21]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])


In [22]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [24]:
df.to_csv("processed_uber_data.csv", index=False)


In [25]:
def etl_pipeline(file_path):
    # Load data
    df = pd.read_csv(file_path)

    # Handle missing values
    numeric_cols = df.select_dtypes(include=["number"]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

    categorical_cols = ["store_and_fwd_flag", "payment_type", "VendorID", "RatecodeID"]
    for col in categorical_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    # Convert datetime columns
    df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
    df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

    # Encode categorical variables
    encoder = LabelEncoder()
    for col in categorical_cols:
        df[col] = encoder.fit_transform(df[col])

    # Scale numerical data
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Save processed data
    df.to_csv("processed_uber_data.csv", index=False)

    print("ETL Process Completed: Processed data saved as 'processed_uber_data.csv'")
    return df


In [26]:
etl_pipeline("uber_data.csv")


ETL Process Completed: Processed data saved as 'processed_uber_data.csv'


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,-2.750778,2016-03-01 00:00:00,2016-03-01 00:07:55,-0.584604,-0.138882,-0.097010,0.099947,-0.141150,0,-0.099344,0.093393,-0.701708,-0.363921,1.970199,0.070302,0.067773,-0.240494,0.030285,-0.279884
1,-2.750778,2016-03-01 00:00:00,2016-03-01 00:11:06,-0.584604,-0.034903,-0.097960,0.100658,-0.141150,0,-0.099585,0.090012,-0.701708,-0.192769,1.970199,0.070302,0.449656,-0.240494,0.030285,-0.072087
2,0.363534,2016-03-01 00:00:00,2016-03-01 00:31:06,0.044564,4.404998,-0.069544,0.069101,-0.141150,0,-0.095076,0.075042,-0.701708,3.529788,1.970199,0.070302,2.339981,-0.240494,0.030285,3.283849
3,0.363534,2016-03-01 00:00:00,2016-03-01 00:00:00,0.673732,2.013482,-0.081025,0.101142,-0.141150,0,-0.094374,0.096429,-0.701708,1.561539,-0.503030,0.070302,0.728431,3.385752,0.030285,1.747530
4,0.363534,2016-03-01 00:00:00,2016-03-01 00:00:00,1.932068,7.121449,-0.096304,0.106875,6.895244,0,-0.124172,0.080071,-0.701708,7.252345,-0.503030,-11.646660,-0.715089,9.905140,0.030285,6.747146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,-2.750778,2016-03-01 06:17:10,2016-03-01 06:22:15,-0.584604,-0.658777,-0.099006,0.096196,-0.141150,0,-0.098480,0.094523,1.375766,-0.706225,-0.503030,0.070302,-0.715089,-0.240494,0.030285,-0.733576
99996,-2.750778,2016-03-01 06:17:10,2016-03-01 06:32:41,-0.584604,0.095070,-0.102334,0.087937,-0.141150,0,-0.096199,0.095062,-0.701708,0.063959,-0.503030,0.070302,0.048678,-0.240494,0.030285,0.028349
99997,-2.750778,2016-03-01 06:17:10,2016-03-01 06:37:23,-0.584604,1.732739,-0.095124,0.102240,-0.141150,0,-0.079351,0.099753,-0.701708,1.347599,-0.503030,0.070302,2.645488,3.385752,0.030285,1.922080
99998,0.363534,2016-03-01 06:17:10,2016-03-01 06:22:09,-0.584604,-0.549599,-0.098160,0.099424,-0.141150,0,-0.094524,0.096789,-0.701708,-0.663437,1.970199,0.070302,-0.195727,-0.240494,0.030285,-0.570109


### Save Processed Data

In [27]:
processed_df = pd.read_csv("processed_uber_data.csv")
processed_df.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,-2.750778,2016-03-01 00:00:00,2016-03-01 00:07:55,-0.584604,-0.138882,-0.097010,0.099947,-0.141150,0,-0.099344,0.093393,-0.701708,-0.363921,1.970199,0.070302,0.067773,-0.240494,0.030285,-0.279884
1,-2.750778,2016-03-01 00:00:00,2016-03-01 00:11:06,-0.584604,-0.034903,-0.097960,0.100658,-0.141150,0,-0.099585,0.090012,-0.701708,-0.192769,1.970199,0.070302,0.449656,-0.240494,0.030285,-0.072087
2,0.363534,2016-03-01 00:00:00,2016-03-01 00:31:06,0.044564,4.404998,-0.069544,0.069101,-0.141150,0,-0.095076,0.075042,-0.701708,3.529788,1.970199,0.070302,2.339981,-0.240494,0.030285,3.283849
3,0.363534,2016-03-01 00:00:00,2016-03-01 00:00:00,0.673732,2.013482,-0.081025,0.101142,-0.141150,0,-0.094374,0.096429,-0.701708,1.561539,-0.503030,0.070302,0.728431,3.385752,0.030285,1.747530
4,0.363534,2016-03-01 00:00:00,2016-03-01 00:00:00,1.932068,7.121449,-0.096304,0.106875,6.895244,0,-0.124172,0.080071,-0.701708,7.252345,-0.503030,-11.646660,-0.715089,9.905140,0.030285,6.747146
